In [1]:
import pandas as pd
import re
import os

In [2]:
df = pd.read_csv('../Data/pbp-2023.csv')
print(df.columns)

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'Unnamed: 10',
       'SeriesFirstDown', 'Unnamed: 12', 'NextScore', 'Description', 'TeamWin',
       'Unnamed: 16', 'Unnamed: 17', 'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsSack', 'IsChallenge', 'IsChallengeReversed',
       'Challenger', 'IsMeasurement', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards'],
      dtype='object')


In [3]:
dropped_columns = ['Unnamed: 10', 'Unnamed: 12', 'Unnamed: 16', 'Unnamed: 17', 'IsMeasurement']
df = df.drop(columns = dropped_columns)

In [4]:
teams = []

for team in df['OffenseTeam']:
    if team not in teams:
        teams.append(team)
        

In [5]:
dates = (df['GameDate'].unique())
dates = pd.to_datetime(dates)
dates = sorted(dates)

In [6]:
def get_week(dates):
    week_numbers = []
    week_num = 1

    # Initialize week_start to the Thursday before the first game
    week_start = dates[0] - pd.Timedelta(days=(dates[0].weekday() + 4) % 7)

    for date in dates:
        # Check if we need to start a new week
        if date - week_start >= pd.Timedelta(days=7):  
            week_num += 1
            week_start = date - pd.Timedelta(days=(date.weekday() + 4) % 7)  # Adjust to the new Thursday

        date_str = date.strftime('%Y-%m-%d')
        week_numbers.append((date_str, week_num))

    return week_numbers

In [7]:
dates_with_weeks = get_week(dates)
prev_week = dates_with_weeks[0][1]

In [8]:
def print_week(week):
    print('week', week)
    return week + 1

def print_bye(week, bye_arr):
    bye_teams = list(set(teams) - set(bye_arr))
    print('BYE WEEKS IN WEEK ', week , bye_teams)

    return []

def print_game_dates(date_arr):
    print(date_arr)
    return []

In [9]:
current_week = 1

print_week(current_week)
week_teams = []
week_dates = []
for date in dates_with_weeks:
    
    if date[1] is not current_week:
        week_teams = print_bye(current_week, week_teams)
        current_week = print_week(current_week)
        week_dates = print_game_dates(week_dates)
    
    games_on_date = df[df['GameDate'] == date[0]]
    week_dates.append(date[0])

    for _, row in games_on_date.iterrows():
        if row['OffenseTeam'] and row['OffenseTeam'] not in week_teams:
            offense_team = row['OffenseTeam']
            defense_team = row['DefenseTeam']

            week_teams.append(offense_team)
            week_teams.append(defense_team)

            print(offense_team , ' vs ' , defense_team)
    


week 1
KC  vs  DET
LA  vs  SEA
NE  vs  PHI
DAL  vs  NYG
MIA  vs  LAC
GB  vs  CHI
PIT  vs  SF
DEN  vs  LV
MIN  vs  TB
WAS  vs  ARI
CLE  vs  CIN
NO  vs  TEN
JAX  vs  IND
BAL  vs  HOU
ATL  vs  CAR
BUF  vs  NYJ
BYE WEEKS IN WEEK  1 []
week 1
['2023-09-07', '2023-09-10', '2023-09-11']
PHI  vs  MIN
NE  vs  MIA
WAS  vs  DEN
LA  vs  SF
ARI  vs  NYG
DAL  vs  NYJ
LAC  vs  TEN
CHI  vs  TB
KC  vs  JAX
GB  vs  ATL
HOU  vs  IND
CIN  vs  BAL
BUF  vs  LV
SEA  vs  DET
PIT  vs  CLE
NO  vs  CAR
BYE WEEKS IN WEEK  2 []
week 2
['2023-09-14', '2023-09-17', '2023-09-18']
NYG  vs  SF
CAR  vs  SEA
ARI  vs  DAL
PIT  vs  LV
KC  vs  CHI
MIN  vs  LAC
DEN  vs  MIA
BUF  vs  WAS
NE  vs  NYJ
CLE  vs  TEN
JAX  vs  HOU
NO  vs  GB
ATL  vs  DET
BAL  vs  IND
LA  vs  CIN
TB  vs  PHI
BYE WEEKS IN WEEK  3 []
week 3
['2023-09-21', '2023-09-24', '2023-09-25']
GB  vs  DET
SF  vs  ARI
KC  vs  NYJ
DAL  vs  NE
LV  vs  LAC
TEN  vs  CIN
PHI  vs  WAS
LA  vs  IND
HOU  vs  PIT
CHI  vs  DEN
CLE  vs  BAL
TB  vs  NO
JAX  vs  ATL
CAR  vs  M

In [13]:
plays = df[df['GameDate']== dates_with_weeks[0][0]]

In [11]:
print(plays.columns)

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'SeriesFirstDown',
       'NextScore', 'Description', 'TeamWin', 'SeasonYear', 'Yards',
       'Formation', 'PlayType', 'IsRush', 'IsPass', 'IsIncomplete',
       'IsTouchdown', 'PassType', 'IsSack', 'IsChallenge',
       'IsChallengeReversed', 'Challenger', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards'],
      dtype='object')


In [26]:
cmp = 0

for index, play in df.iterrows():
    if (
        play['OffenseTeam'] == 'DAL' 
        and play['IsPass'] == 1 
        and play['IsNoPlay'] == 0 
        and play['IsIncomplete'] == 0 
        and play['IsInterception'] == 0 
        and play['IsFumble'] == 0 
        and play['PenaltyYards'] == 0
        ):
        cmp += play['Yards']

print(cmp)


4543


In [238]:
third_down_attempts = 0
play_arr = []

for index,play in plays.iterrows():
    if play['OffenseTeam'] == 'GB' and play['IsNoPlay'] == 0 and play['Down'] == 3 and (play['IsPass'] == 1 or play['IsRush'] == 1):
        third_down_attempts += 1


print(third_down_attempts)

16


In [ ]:
def verify_play(play):
    if play['IsNoPlay'] == 0 and play['Down'] == 3 and (play['IsTouchdown'] == 1 or play['IsRush'] == 1 or play['IsPass'] == 1)

In [245]:
print(df['PlayType'].unique())

['RUSH' 'CLOCK STOP' 'FIELD GOAL' nan 'KICK OFF' 'PASS' 'SACK' 'TIMEOUT'
 'SCRAMBLE' 'NO PLAY' 'TWO-POINT CONVERSION' 'EXTRA POINT' 'QB KNEEL'
 'FUMBLES' 'PUNT' 'EXCEPTION']


In [244]:
third_down = {team: 0 for team in teams}


attempts = 0
plays_arr = []

for index, play in df[df['OffenseTeam'] == 'CLE'].iterrows():
    if play['IsNoPlay'] == 0 and play['Down'] == 3 and (play['IsTouchdown'] == 1 or play['IsRush'] == 1 or play['IsPass'] == 1):
        attempts += 1

print(attempts)

232


In [252]:
print(plays['PlayType'].unique())

['RUSH' 'KICK OFF' 'FIELD GOAL' 'PASS' nan 'EXTRA POINT' 'SACK'
 'TWO-POINT CONVERSION' 'TIMEOUT' 'SCRAMBLE' 'QB KNEEL' 'NO PLAY'
 'FUMBLES' 'PUNT' 'CLOCK STOP']


In [262]:
play_arr = []

for index, play in df.iterrows():
    if pd.isna(play['PlayType']) and not (
        play['Description'].startswith("TIMEOUT") or 
        play['Description'] == 'TWO-MINUTE WARNING' or 
        play['Description'].startswith('END')
    ):
        play_arr.append(play)

play_df = pd.DataFrame(play_arr)
play_df.to_csv('PLAYS.csv', index=False)


In [272]:
# Fix the issue with dataset that direct snaps are not correct

def extract_yardage(play_description):
    match = re.search(r'FOR (-?\d+) YARDS?', play_description)
    if match:
        return int(match.group(1))
    else:
        return 0



In [273]:
for index, play in play_df.iterrows():
    yards = extract_yardage(play['Description'])
    print(play['Description'] , " : ", yards)

(:56) (SHOTGUN) DIRECT SNAP TO 2-A.MATTISON.  2-A.MATTISON RIGHT GUARD TO DEN 4 FOR 8 YARDS (93-D.JONES; 31-J.SIMMONS).  :  8
(5:02) (SHOTGUN) DIRECT SNAP TO 1-T.ETIENNE.  1-T.ETIENNE RIGHT GUARD TO CLE 1 FOR NO GAIN (33-R.HICKMAN, 22-G.DELPIT). CLE-29-C.MITCHELL WAS INJURED DURING THE PLAY. HIS RETURN IS PROBABLE.  JACKSONVILLE CHALLENGED THE SHORT OF THE GOAL LINE RULING, AND THE PLAY WAS REVERSED. (SHOTGUN) DIRECT SNAP TO 1-T.ETIENNE.  1-T.ETIENNE RIGHT GUARD FOR 1 YARD, TOUCHDOWN. CLE-29-C.MITCHELL WAS INJURED DURING THE PLAY. HIS RETURN IS PROBABLE.  :  1
(10:13) (SHOTGUN) DIRECT SNAP TO 33-J.WILLIAMS.  33-J.WILLIAMS RIGHT GUARD FOR 2 YARDS, TOUCHDOWN.  :  2
(7:40) (SHOTGUN) DIRECT SNAP TO 10-I.PACHECO.  10-I.PACHECO LEFT END FOR 12 YARDS, TOUCHDOWN.  :  12
(3:58) (SHOTGUN) DIRECT SNAP TO 27-K.HUNT.  27-K.HUNT RIGHT TACKLE TO HOU 9 FOR 3 YARDS (41-M.SANDERS, 25-D.KING).  :  3
(9:46) (SHOTGUN) 68-M.DUNN AND 69-L.WATSON REPORTED IN AS ELIGIBLE. DIRECT SNAP TO 27-K.HUNT.  27-K.HUNT L

**1st Down**
- Eff : A play that gains 3 or more yards

**2nd Down**
- Eff : A play that gains 50% or more of the yards to go
- Short : Distance <= 3 yards
- Medium : Distance 4-7 yards
- Long : Distance > 7 yards

**3rd Down**
- Eff : A play that gains the first down
- Short : Distance <= 3 yards
- Medium : Distance 4-7 yards
- Long : Distance > 7 yards

In [19]:
general_schema = [
    "1_pass_att",
    "1_rush_att",
    "1_pass_eff",
    "1_rush_eff",
    "1_pass_avg",
    "1_rush_avg",
    
    "2_long_pass_att",
    "2_long_rush_att",
    "2_long_pass_eff",
    "2_long_rush_eff",
    "2_long_pass_avg",
    "2_long_rush_avg",

    "2_med_pass_att",
    "2_med_rush_att",
    "2_med_pass_eff",
    "2_med_rush_eff",
    "2_med_pass_avg",
    "2_med_rush_avg",

    "2_short_pass_att",
    "2_short_rush_att",
    "2_short_pass_eff",
    "2_short_rush_eff",
    "2_short_pass_avg",
    "2_short_rush_avg",

    "3_long_pass_att",
    "3_long_rush_att",
    "3_long_pass_eff",
    "3_long_rush_eff",
    "3_long_pass_avg",
    "3_long_rush_avg",

    "3_med_pass_att",
    "3_med_rush_att",
    "3_med_pass_eff",
    "3_med_rush_eff",
    "3_med_pass_avg",
    "3_med_rush_avg",

    "3_short_pass_att",
    "3_short_rush_att",
    "3_short_pass_eff",
    "3_short_rush_eff",
    "3_short_pass_avg",
    "3_short_rush_avg",
]

SyntaxError: unterminated string literal (detected at line 7) (1464103997.py, line 7)

In [ ]:
quarter_schema = [
    
]

In [ ]:
situational_schema = [
    "2_min_half_run",
    "2_min_half_pass",

]